In [1]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
import sys
if ".." not in sys.path:
    sys.path.append("..")
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# CHAPITRE 8 - Bases de données

## III. Création d'une base de données

SQLite est un système de gestion de base de données relationnelle très répandu. Noter qu'il existe d'autres systèmes de gestion de base de données relationnelle comme MySQL ou PostgreSQL. Dans tous les cas, le langage de requête utilisé est le SQL (même si parfois on peut noter quelques petites différences). Ce qui sera vu ici avec SQLite pourra, à quelques petites modifications près, être utilisé avec, par exemple, MySQL.

Nous allons commencer par créer notre base de données :

In [4]:
import sqlite3
connection = sqlite3.connect("mabase.db")
cursor = connection.cursor()

pour l'instant elle ne contient aucune table (aucune relation), pour créer une table, on utilise la requête:

In [5]:
# Création de la table livre
cursor.execute("""
CREATE TABLE LIVRES
(id INT, titre TEXT, id_auteur INT, ann_publi INT, note INT);
""")

On peut vérifier qu'elle est bien créée et qu'elle a la bonne structure.

In [6]:
for row in cursor.execute("PRAGMA table_info(livres);"):
    print(row)

(0, 'id', 'INT', 0, None, 0)
(1, 'titre', 'TEXT', 0, None, 0)
(2, 'id_auteur', 'INT', 0, None, 0)
(3, 'ann_publi', 'INT', 0, None, 0)
(4, 'note', 'INT', 0, None, 0)


Pour l'instant cette table est vide

In [7]:
for row in cursor.execute("SELECT * FROM livres;"):
    print(row)

On va créer aussi une table **auteurs**

In [8]:
cursor.execute("""CREATE TABLE AUTEURS
(id INT, nom TEXT, prenom TEXT, ann_naissance INT, langue_ecriture TEXT);
""")

In [9]:
for row in cursor.execute("PRAGMA table_info(auteurs);"):
    print(row)

(0, 'id', 'INT', 0, None, 0)
(1, 'nom', 'TEXT', 0, None, 0)
(2, 'prenom', 'TEXT', 0, None, 0)
(3, 'ann_naissance', 'INT', 0, None, 0)
(4, 'langue_ecriture', 'TEXT', 0, None, 0)


Pour remplir ces tables, on peut utiliser les requêtes suivantes:

In [10]:
cursor.execute("""INSERT INTO AUTEURS
(id,nom,prenom,ann_naissance,langue_ecriture)
VALUES
(1,'Orwell','George',1903,'anglais'),
(2,'Herbert','Frank',1920,'anglais'),
(3,'Asimov','Isaac',1920,'anglais'),
(4,'Huxley','Aldous',1894,'anglais'),
(5,'Bradbury','Ray',1920,'anglais'),
(6,'K.Dick','Philip',1928,'anglais'),
(7,'Barjavel','René',1911,'français'),
(8,'Boulle','Pierre',1912,'français'),
(9,'Van Vogt','Alfred Elton',1912,'anglais'),
(10,'Verne','Jules',1828,'français');""")

In [11]:
cursor.execute("""INSERT INTO LIVRES
(id,titre,id_auteur,ann_publi,note)
VALUES
(1,'1984',1,1949,10),
(2,'Dune',2,1965,8),
(3,'Fondation',3,1951,9),
(4,'Le meilleur des mondes',4,1931,7),
(5,'Fahrenheit 451',5,1953,7),
(6,'Ubik',6,1969,9),
(7,'Chroniques martiennes',5,1950,8),
(8,'La nuit des temps',7,1968,7),
(9,'Blade Runner',6,1968,8),
(10,'Les Robots',3,1950,9),
(11,'La Planète des singes',8,1963,8),
(12,'Ravage',7,1943,8),
(13,'Le Maître du Haut Château',6,1962,8),
(14,'Le monde des Ā',9,1945,7),
(15,"La Fin de l'éternité",3,1955,8),
(16,'De la Terre à la Lune',10,1865,10);""")

## Exercice 1
1. Que font les requêtes suivantes ?

In [12]:
for row in cursor.execute("""SELECT titre FROM livres
WHERE ann_publi> 1960"""):
    print(row)

('Dune',)
('Ubik',)
('La nuit des temps',)
('Blade Runner',)
('La Planète des singes',)
('Le Maître du Haut Château',)


In [13]:
for row in cursor.execute("""SELECT * FROM auteurs
WHERE nom = 'Asimov'"""):
    print(row)

(3, 'Asimov', 'Isaac', 1920, 'anglais')


In [14]:
for row in cursor.execute("""SELECT * FROM livres
JOIN auteurs ON livres.id_auteur = auteurs.id"""):
    print(row)

(1, '1984', 1, 1949, 10, 1, 'Orwell', 'George', 1903, 'anglais')
(2, 'Dune', 2, 1965, 8, 2, 'Herbert', 'Frank', 1920, 'anglais')
(3, 'Fondation', 3, 1951, 9, 3, 'Asimov', 'Isaac', 1920, 'anglais')
(4, 'Le meilleur des mondes', 4, 1931, 7, 4, 'Huxley', 'Aldous', 1894, 'anglais')
(5, 'Fahrenheit 451', 5, 1953, 7, 5, 'Bradbury', 'Ray', 1920, 'anglais')
(6, 'Ubik', 6, 1969, 9, 6, 'K.Dick', 'Philip', 1928, 'anglais')
(7, 'Chroniques martiennes', 5, 1950, 8, 5, 'Bradbury', 'Ray', 1920, 'anglais')
(8, 'La nuit des temps', 7, 1968, 7, 7, 'Barjavel', 'René', 1911, 'français')
(9, 'Blade Runner', 6, 1968, 8, 6, 'K.Dick', 'Philip', 1928, 'anglais')
(10, 'Les Robots', 3, 1950, 9, 3, 'Asimov', 'Isaac', 1920, 'anglais')
(11, 'La Planète des singes', 8, 1963, 8, 8, 'Boulle', 'Pierre', 1912, 'français')
(12, 'Ravage', 7, 1943, 8, 7, 'Barjavel', 'René', 1911, 'français')
(13, 'Le Maître du Haut Château', 6, 1962, 8, 6, 'K.Dick', 'Philip', 1928, 'anglais')
(14, 'Le monde des Ā', 9, 1945, 7, 9, 'Van Vogt

In [15]:
for row in cursor.execute("""SELECT titre FROM livres
JOIN auteurs ON livres.id_auteur = auteurs.id
WHERE auteurs.nom = 'Asimov'"""):
    print(row)

('Fondation',)
("La Fin de l'éternité",)
('Les Robots',)


Comme nous avons créé cette base de données, nous en somme propriétaire et nous pouvons insérer un nouvel enregistrement et également en supprimer.

Pour insérer un nouvel auteur :

In [16]:
cursor.execute("""INSERT INTO AUTEURS
(id,nom,prenom,ann_naissance,langue_ecriture)
VALUES
(11,'Simmons','Dan',1948,'anglais')""")

Et pour insérer un nouveau livre :

In [17]:
cursor.execute("""INSERT INTO LIVRES
(id,titre,id_auteur,ann_publi,note)
VALUES
(17,'Hypérion',11,1991,10)""")

Pour modifier un enregistrement on utilise le mot clé **UPDATE**:

In [18]:
cursor.execute("""UPDATE LIVRES
SET note = 7
WHERE titre = 'Hypérion'""")

Et enfin pour supprimer une entrée:

In [19]:
cursor.execute("""DELETE FROM LIVRES
WHERE titre='Hypérion'""")

## Exercice 2
Écrire des requêtes pour :

1. attribuer la note de 10 à tous les livres écrits par Asimov publiés après 1950

2. supprimer les livres publiés avant 1945



In [23]:
cursor.execute("""UPDATE LIVRES
SET note = 10
WHERE id_auteur = (SELECT id FROM auteurs where nom = 'Azimov') AND ann_publi > 1950""")

In [22]:
cursor.execute("""DELETE FROM LIVRES
WHERE ann_publi < 1945""")

In [25]:
for row in cursor.execute("""
SELECT * FROM livres;
"""):
    print(row)

(1, '1984', 1, 1949, 10)
(2, 'Dune', 2, 1965, 8)
(3, 'Fondation', 3, 1951, 10)
(5, 'Fahrenheit 451', 5, 1953, 7)
(6, 'Ubik', 6, 1969, 9)
(7, 'Chroniques martiennes', 5, 1950, 8)
(8, 'La nuit des temps', 7, 1968, 7)
(9, 'Blade Runner', 6, 1968, 8)
(10, 'Les Robots', 3, 1950, 10)
(11, 'La Planète des singes', 8, 1963, 8)
(13, 'Le Maître du Haut Château', 6, 1962, 8)
(14, 'Le monde des Ā', 9, 1945, 7)
(15, "La Fin de l'éternité", 3, 1955, 10)


## Exercice 3
1. Créer une base de données "mon_entreprise.db" dont le schéma relationnel est ![schema](bdd4_cours08.png)
2. Ajouter des enregistrements dans chaque table
3. Faire afficher la ou les commandes d'un client.

In [5]:
connection = sqlite3.connect("mon_entreprise.db")
cursor = connection.cursor()

In [6]:
cursor.execute("""
CREATE TABLE CLIENTS
(id INT, nom TEXT, prenom TEXT, n_tel INT, adresse TEXT)
""")

In [7]:
cursor.execute("""
CREATE TABLE ARTICLES
(code TEXTE, nom TEXT, description INT, prix INT);
""")

In [8]:
cursor.execute("""
CREATE TABLE COMANDES
(numero INT, id_client INT, id_article TEXTE, quantite INT);
""")

In [9]:
cursor.execute("""
INSERT INTO CLIENTS
(id, nom, prenom, n_tel, adresse)
VALUES
(1, 'Edou', 'Jean', 0613543709, '19 Rue de la mort'),
(2, 'Terma', 'Bertrand', 0754235912, '22 Avenue Moulin')
""")

In [10]:
cursor.execute("""
INSERT INTO ARTICLES
(code , nom , description , prix )
VALUES
('A', 'chaise_enfant', 'petite chaise', 19),
('B', 'canaper', 'canaper volumineux', 345)
""")

In [11]:
cursor.execute("""
INSERT INTO COMANDES
(numero, id_client, id_article, quantite)
VALUES
(1,2,'A',12),
(2,1,'B',3)
""")

In [16]:
for row in cursor.execute("""
SELECT * FROM comandes 
JOIN clients ON clients.id = comandes.id_client
JOIN articles ON articles.code = comandes.id_article
WHERE clients.nom = 'Edou';
"""):
    print(row)

(2, 1, 'B', 3, 1, 'Edou', 'Jean', 613543709, '19 Rue de la mort', 'B', 'canaper', 'canaper volumineux', 345)


In [18]:
for row in cursor.execute("""
SELECT articles.nom, clients.nom, quantite*prix AS prix_total FROM comandes 
JOIN clients ON clients.id = comandes.id_client
JOIN articles ON articles.code = comandes.id_article
;
"""):
    print(row)

('chaise_enfant', 'Terma', 228)
('canaper', 'Edou', 1035)
